In [29]:
from dataclasses import dataclass, field
from typing import Tuple, List, NamedTuple, FrozenSet, Set
from parse import parse
from collections import Counter, defaultdict, deque
from math import prod, inf
from heapq import heappop, heappush
from functools import lru_cache, cache
from itertools import combinations, permutations, cycle


import string
import hashlib
import re
import numpy as np

import black
import jupyter_black


jupyter_black.load(lab=True, target_version=black.TargetVersion.PY310)

In [2]:
# Day 1: No Time for a Taxicab
@dataclass
class Position:
    heading: int
    x: int
    y: int
    visited: set = field(default_factory=set)
    double_visit_distance: int = 0

    # NESW = 0, 1, 2, 3
    HEADINGS = ((0, 1), (1, 0), (0, -1), (-1, 0))

    def forward(self, steps: int):
        "Move forward `steps` given current heading"
        for _ in range(steps):
            self.x = self.x + self.HEADINGS[self.heading][0]
            self.y = self.y + self.HEADINGS[self.heading][1]

            if not self.double_visit_distance:
                if (self.x, self.y) in self.visited:
                    self.double_visit_distance = abs(self.x) + abs(self.y)
                self.visited.add((self.x, self.y))

    def turn_right(self):
        self.heading = (self.heading + 1) % 4

    def turn_left(self):
        self.heading = (self.heading - 1) % 4


directions = [(dir[0], int(dir[1:])) for dir in open("2016/1.txt").read().split(", ")]

pos = Position(0, 0, 0)
for turn, steps in directions:
    if turn == "R":
        pos.turn_right()
    elif turn == "L":
        pos.turn_left()
    else:
        raise ValueError(dir)
    pos.forward(steps)

print(f"Part 1: {abs(pos.x) + abs(pos.y)}")  # 236
print(f"Part 2: {pos.double_visit_distance}")  # 182

Part 1: 236
Part 2: 182


In [6]:
# Day 2: Bathroom Security
@dataclass
class Keypad:
    """
    Implements a keypad where you can move up, down, right, and left. You can't move past
    an edge. Represents current value as a string of this object.

    Keypad looks like this:

    1 2 3
    4 5 6
    7 8 9
    """

    number: int = 5

    def up(self):
        if self.number not in (1, 2, 3):
            self.number -= 3

    def down(self):
        if self.number not in (7, 8, 9):
            self.number += 3

    def left(self):
        if self.number not in (1, 4, 7):
            self.number -= 1

    def right(self):
        if self.number not in (3, 6, 9):
            self.number += 1

    def __str__(self):
        return str(self.number)


@dataclass
class BathroomKeypad:
    """
    Keypad looks like below.

        1
      2 3 4
    5 6 7 8 9
      A B C
        D
    """

    number: int = 5
    # Internal representation of keypad
    #       1
    #    2  3  4
    # 5  6  7  8  9
    #   10 11 12
    #      13

    def up(self):
        if self.number not in (5, 2, 1, 4, 9):
            if self.number in (3, 13):
                self.number -= 2
            elif self.number in (6, 7, 8, 10, 11, 12):
                self.number -= 4

    def down(self):
        if self.number not in (5, 10, 13, 12, 9):
            if self.number in (1, 11):
                self.number += 2
            else:
                self.number += 4

    def left(self):
        if self.number not in (1, 2, 5, 10, 13):
            self.number -= 1

    def right(self):
        if self.number not in (1, 4, 9, 12, 13):
            self.number += 1

    def __str__(self) -> str:
        return hex(self.number)[-1]


lines = open("2016/2.txt").readlines()

a = b = ""
keypad = Keypad()
bathroom = BathroomKeypad()
for line in lines:
    for move in line:
        match move:
            case "U":
                keypad.up()
                bathroom.up()
            case "D":
                keypad.down()
                bathroom.down()
            case "L":
                keypad.left()
                bathroom.left()
            case "R":
                keypad.right()
                bathroom.right()
    a += str(keypad)
    b += str(bathroom)
print(f"Part 1: {a}")
print(f"Part 1: {b}")

Part 1: 52981
Part 1: 74cd2


In [220]:
# Day 3: Squares With Three Sides
def possible(sides: Tuple[int, int, int]) -> bool:
    "Returns true if this combination is possible."
    a, b, c = sorted(sides)
    return a + b > c


lines = open("2016/3.txt").readlines()
lines = [line.strip().split() for line in lines]

sides = [tuple(int(side) for side in line) for line in lines]
print(f"Part 1: {sum(possible(side) for side in sides)}")  # 869

sides = [side for i in range(0, len(sides), 3) for side in zip(*sides[i : i + 3])]
print(f"Part 2: {sum(possible(side) for side in sides)}")  # 1544

Part 1: 869
Part 2: 1544


In [238]:
# Day 4: Security Through Obscurity
def real_room(name: str, checksum: str) -> bool:
    cnt = Counter(sorted(name.replace("-", "")))
    chk = "".join([kv[0] for kv in cnt.most_common(5)])
    return chk == checksum


def decrypt_room(room: str, steps: int) -> str:
    az = string.ascii_lowercase
    shift = steps % len(az)
    tr = str.maketrans(az, az[shift:] + az[:shift])
    return room.translate(tr)


lines = open("2016/4.txt").readlines()
rooms = [parse("{}-{:d}[{}]", line.strip()) for line in lines]
id_sum = 0
for name, sector, checksum in rooms:
    if real_room(name, checksum):
        id_sum += sector
        # Part 2: Sector ID of North Pole objects
        if "north" in decrypt_room(name, sector):
            print(f"Part 2: {sector}: {decrypt_room(name, sector)}")  # 482
print(f"Part 1: {id_sum}")

Part 2: 482: northpole-object-storage
Part 1: 361724


In [2]:
# Day 5: How About a Nice Game of Chess
# 10 s execution time part 1, 40 s when including part 2
base = "abbhdwsy"
pass1 = []
pass2 = [0] * 8
index = 0
while not all(pass2):
    candidate = base + str(index)
    hash = hashlib.md5(candidate.encode("utf-8")).hexdigest()
    if hash.startswith("00000"):
        if len(pass1) < 8:
            pass1.append(hash[5])
        pos = int(hash[5], base=16)
        if pos < 8 and not pass2[pos]:
            pass2[pos] = hash[6]
    index += 1
print(f'Part 1: {"".join(pass1[:8])}')
print(f'Part 2: {"".join(pass2)}')

Part 1: 801b56a7
Part 2: 424a0197


In [97]:
# Day 6: Signals and Noise
lines = open("2016/6.txt").read()
messages = [line.strip() for line in lines.splitlines()]
counters = [Counter(column) for column in (zip(*messages))]
print(f'Part 1: {"".join(c.most_common(1)[0][0] for c in counters)}') # tsreykjj
print(f'Part 2: {"".join(c.most_common()[-1][0][0] for c in counters)}') # hnfbujie

Part 1: tsreykjj
Part 2: hnfbujie


In [98]:
# Day 7: Internet Protocol Version 7
def abba(text: str, pos: int) -> bool:
    if text[pos] == text[pos + 3] and text[pos + 1] == text[pos + 2]:
        if text[pos] != text[pos + 1]:
            return True


def supports_tls(text: str) -> bool:
    abba_outside = False
    outside_brackets = True
    for i in range(len(text) - 3):
        if text[i] == "[":
            outside_brackets = False
            next
        if text[i] == "]":
            outside_brackets = True
            next
        if abba(text, i):
            if outside_brackets:
                abba_outside = True
            else:
                return False
    return abba_outside


assert supports_tls("abba[mnop]qrst") == True
assert supports_tls("abcd[bddb]xyyx") == False
assert supports_tls("aaaa[qwer]tyui") == False
assert supports_tls("ioxxoj[asdfgh]zxcvbn") == True


def aba_outside(text: str) -> str:
    supernet = True
    for a, b, c in zip(text, text[1:], text[2:]):
        if a == "[":
            supernet = False
            next
        if a == "]":
            supernet = True
            next
        if supernet and a == c and a != b:
            yield a + b + a


def bab_inside(text: str, candidate: str) -> bool:
    hypernet = False
    for a, b, c in zip(text, text[1:], text[2:]):
        if a == "[":
            hypernet = True
            next
        if a == "]":
            hypernet = False
            next
        if hypernet and a == c and candidate == b + a + b:
            return True


def supports_sls(text: str) -> bool:
    return any(bab_inside(text, candidate) for candidate in aba_outside(text))


assert supports_sls("aba[bab]xyz") == True
assert supports_sls("xyx[xyx]xyx") == False
assert supports_sls("aaa[kek]eke") == True
assert supports_sls("zazbz[bzb]cdb") == True

lines = open("2016/7.txt").read()
lines = [line for line in lines.splitlines()]
print(f"Part 1: {sum(supports_tls(line) for line in lines)}")  # 115
print(f"Part 2: {sum(supports_sls(line) for line in lines)}")  # 231

Part 1: 115
Part 2: 231


In [36]:
# Day 8: Two-Factor Authentication
lines = open("2016/8.txt").read().splitlines()
commands = [[line] + [int(x) for x in re.findall(r"[\d]+", line)] for line in lines]
display = [[0] * 50 for _ in range(6)]


def turn_on(display, wide: int, tall: int):
    for r in range(tall):
        for c in range(wide):
            display[r][c] = 1
    return display


def rotate_row(display, r: int, steps: int):
    "Rotate row `r` to the right"
    row = display[r]
    display[r] = row[len(row) - steps :] + row[: len(row) - steps]
    return display


def rotate_column(display, c: int, steps: int):
    "Rotate column `c` down"
    transposed = list(zip(*display))
    transposed = rotate_row(transposed, c, steps)
    display = list(zip(*transposed))
    return [list(row) for row in display]


def print_display(display):
    for row in range(len(display)):
        for col in range(len(display[row])):
            if display[row][col]:
                print("##", end="")
            else:
                print("  ", end="")
        print()


for command, a, b in commands:
    if command.startswith("rect"):
        display = turn_on(display, wide=a, tall=b)
    if command.startswith("rotate row"):
        display = rotate_row(display, r=a, steps=b)
    if command.startswith("rotate column"):
        display = rotate_column(display, c=a, steps=b)
print(f"Part 1: {sum(val for line in display for val in line)}")
print_display(display)

Part 1: 123
  ####    ########  ######    ##    ##  ######    ########  ######        ####  ######      ######  
##    ##  ##        ##    ##  ##    ##  ##    ##        ##  ##    ##        ##  ##    ##  ##        
##    ##  ######    ######    ##    ##  ##    ##      ##    ######          ##  ##    ##  ##        
########  ##        ##    ##  ##    ##  ######      ##      ##    ##        ##  ######      ####    
##    ##  ##        ##    ##  ##    ##  ##        ##        ##    ##  ##    ##  ##              ##  
##    ##  ##        ######      ####    ##        ########  ######      ####    ##        ######    


In [5]:
# Day 8 with numpy
lines = open("2016/8.txt").read().splitlines()
commands = [[line] + [int(x) for x in re.findall(r"[\d]+", line)] for line in lines]

display = np.zeros((6, 50), dtype=np.int8)
for command, a, b in commands:
    if command.startswith("rect"):
        display[:b, :a] = 1
    if command.startswith("rotate row"):
        display[a] = np.roll(display[a], shift=b)
    if command.startswith("rotate column"):
        display[:, a] = np.roll(display[:, a], shift=b)
        # Can also use:
        # display.T[a] = np.roll(display.T[a], shift=b)
print(f"Part 1: {np.sum(display)}")
for row in display:
    print("".join("⬜️🟦"[point * 2 : point * 2 + 2] for point in row))

Part 1: 123
⬜️🟦🟦⬜️⬜️🟦🟦🟦🟦⬜️🟦🟦🟦⬜️⬜️🟦⬜️⬜️🟦⬜️🟦🟦🟦⬜️⬜️🟦🟦🟦🟦⬜️🟦🟦🟦⬜️⬜️⬜️⬜️🟦🟦⬜️🟦🟦🟦⬜️⬜️⬜️🟦🟦🟦⬜️
🟦⬜️⬜️🟦⬜️🟦⬜️⬜️⬜️⬜️🟦⬜️⬜️🟦⬜️🟦⬜️⬜️🟦⬜️🟦⬜️⬜️🟦⬜️⬜️⬜️⬜️🟦⬜️🟦⬜️⬜️🟦⬜️⬜️⬜️⬜️🟦⬜️🟦⬜️⬜️🟦⬜️🟦⬜️⬜️⬜️⬜️
🟦⬜️⬜️🟦⬜️🟦🟦🟦⬜️⬜️🟦🟦🟦⬜️⬜️🟦⬜️⬜️🟦⬜️🟦⬜️⬜️🟦⬜️⬜️⬜️🟦⬜️⬜️🟦🟦🟦⬜️⬜️⬜️⬜️⬜️🟦⬜️🟦⬜️⬜️🟦⬜️🟦⬜️⬜️⬜️⬜️
🟦🟦🟦🟦⬜️🟦⬜️⬜️⬜️⬜️🟦⬜️⬜️🟦⬜️🟦⬜️⬜️🟦⬜️🟦🟦🟦⬜️⬜️⬜️🟦⬜️⬜️⬜️🟦⬜️⬜️🟦⬜️⬜️⬜️⬜️🟦⬜️🟦🟦🟦⬜️⬜️⬜️🟦🟦⬜️⬜️
🟦⬜️⬜️🟦⬜️🟦⬜️⬜️⬜️⬜️🟦⬜️⬜️🟦⬜️🟦⬜️⬜️🟦⬜️🟦⬜️⬜️⬜️⬜️🟦⬜️⬜️⬜️⬜️🟦⬜️⬜️🟦⬜️🟦⬜️⬜️🟦⬜️🟦⬜️⬜️⬜️⬜️⬜️⬜️⬜️🟦⬜️
🟦⬜️⬜️🟦⬜️🟦⬜️⬜️⬜️⬜️🟦🟦🟦⬜️⬜️⬜️🟦🟦⬜️⬜️🟦⬜️⬜️⬜️⬜️🟦🟦🟦🟦⬜️🟦🟦🟦⬜️⬜️⬜️🟦🟦⬜️⬜️🟦⬜️⬜️⬜️⬜️🟦🟦🟦⬜️⬜️


In [68]:
# Day 9: Explosives in Cyberspace
def text_length(text: str, part_one=False) -> int:
    "Expands first marker if part_one and all markers found otherwise. Returns length after expansion."
    match = re.search(r"\((\d+)x(\d+)\)", text)
    if not match:
        return len(text)
    length = int(match[1])
    repetitions = int(match[2])
    head = match.start()
    middle = text[match.end() : match.end() + length]
    tail = text[match.end() + length :]
    if part_one:
        return head + length * repetitions + text_length(tail, part_one)
    else:
        return head + repetitions * text_length(middle) + text_length(tail)


text = open("2016/9.txt").read().strip()

print(f"Part 1: {text_length(text, part_one=True)}")  # 115118
print(f"Part 2: {text_length(text)}")  # 11107527530

Part 1: 115118
Part 2: 11107527530


In [166]:
# Day 10: Balance Bots
@dataclass
class BotRule:
    low_dest: str = ""
    low: int = -1
    high_dest: str = ""
    high: int = -1
    low_value: int = -1
    high_value: int = -1


def parse_lines(lines):
    for line in lines:
        if p := parse("value {:d} goes to bot {:d}", line):
            value, bot = p
            bots[bot].append(value)
        elif p := parse("bot {:d} gives low to {} {:d} and high to {} {:d}", line):
            bot, *rules = p
            bot_rules[bot] = BotRule(*rules)
        else:
            raise ValueError("Can not parse", line)


def process_bot(bot: int):
    assert len(bots[bot]) == 2
    # Update rules
    bot_rule = bot_rules[bot]
    bot_rule.low_value = min(bots[bot])
    bot_rule.high_value = max(bots[bot])
    # Give away chips
    if bot_rule.low_dest == "bot":
        bots[bot_rule.low].append(bot_rule.low_value)
    elif bot_rule.low_dest == "output":
        outputs[bot_rule.low].append(bot_rule.low_value)
    else:
        raise ValueError("Bad rule", bot_rule.low_dest)
    if bot_rule.high_dest == "bot":
        bots[bot_rule.high].append(bot_rule.high_value)
    elif bot_rule.high_dest == "output":
        outputs[bot_rule.high].append(bot_rule.high_value)
    else:
        raise ValueError("Bad rule", bot_rule.high_dest)

    bots[bot] = []


def bot_to_process() -> int:
    for key, value in bots.items():
        if len(value) > 1:
            return key
    return -1


bots = defaultdict(lambda: list())
bot_rules = {}
outputs = defaultdict(lambda: list())
lines = open("2016/10.txt").read().strip().splitlines()
parse_lines(lines)

while (bot := bot_to_process()) != -1:
    process_bot(bot)

for bot, rule in bot_rules.items():
    if rule.low_value == 17 and rule.high_value == 61:
        print(f"Part 1: {bot}")  # 157
print(f"Part 2: {prod(outputs[i][0] for i in (0, 1, 2))}")  # 1085

Part 1: 157
Part 2: 1085


In [75]:
# Day 11: Radioisotope Thermoelectric Generators
class State(NamedTuple):
    "Describes current elevator level, microchips and generators per floor."
    elevator: int
    floors: Tuple[FrozenSet, FrozenSet, FrozenSet, FrozenSet]


def fs(*items):
    return frozenset(sorted(items))


def combos(items):
    "1 and 2 long combinations of `items`."
    for i in (1, 2):
        for x in combinations(items, i):
            yield fs(*x)


def legal_floor(floor):
    """
    The chips are prototypes and don't have normal radiation shielding, but they do have the ability to generate an electromagnetic radiation shield when powered. Unfortunately, they can only be powered by their corresponding RTG. An RTG powering a microchip is still dangerous to other microchips.
    """
    generators = {gen[0] for gen in floor if gen[-1] == "G"}
    microchips = {chip[0] for chip in floor if chip[-1] == "M"}
    if not generators:
        # A floor without generators is safe
        return True
    return all((chip in generators) for chip in microchips)


def moves(state: State):
    legal_floors = {0, 1, 2, 3}
    L, floors = state
    for L2 in {L + 1, L - 1} & legal_floors:
        # bring one or two things to the new floors
        for bring in combos(floors[L]):
            new_floors = list(range(len(legal_floors)))
            for floor in legal_floors:
                if floor == L2:
                    new_floors[floor] = bring | floors[L2]
                elif floor == L:
                    new_floors[floor] = floors[floor] - bring
                else:
                    new_floors[floor] = floors[floor]
            if legal_floor(new_floors[L]) and legal_floor(new_floors[L2]):
                yield State(L2, tuple(new_floors))


def done(state: State) -> bool:
    return state.elevator == 3 and not any(state.floors[i] for i in range(3))


def a_star(state: State):
    def heuristic(state) -> int:
        "An estimate of the number of moves needed to move everything to top."
        total = sum(len(floor) * i for (i, floor) in enumerate(reversed(state.floors)))
        return total // 2  # Can move two items in one move.

    # A star with defaultdict and native heapq instead of PriorityQueue
    frontier = []
    heappush(frontier, (0, state))
    came_from = {state: None}
    cost_so_far = defaultdict(lambda: inf)
    cost_so_far[state] = 0

    while frontier:
        current = heappop(frontier)[1]

        if done(current):
            return cost_so_far[current]

        for next in moves(current):
            new_cost = cost_so_far[current] + 1
            if new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost + heuristic(next)
                heappush(frontier, (priority, next))
                came_from[next] = current


Ø = frozenset()

example = State(0, (fs("HM", "LM"), fs("HG"), fs("LG"), Ø))  # 11

part1 = State(
    0, (fs("PM", "PG"), fs("CG", "cG", "RG", "pG"), fs("CM", "cM", "RM", "pM"), Ø)
)  # 33

part2 = State(
    0,
    (
        fs("EG", "EM", "DG", "DM", "PM", "PG"),
        fs("CG", "cG", "RG", "pG"),
        fs("CM", "cM", "RM", "pM"),
        Ø,
    ),
) # 57

print("Part 1:", a_star(part1))  # Runs in 10 seconds
print("Part 2:", a_star(part2))  # 10 minutes

Part 1: 33
Part 2: 57


In [1]:
# Day 12: Leonardo's Monorail
def value(x):
    try:
        return int(x)
    except ValueError:
        return reg[x]


pc = 0
reg = {"a": 0, "b": 0, "c": 0, "d": 0}
# reg["c"] = 1 # Uncomment to run part 2, takes 30s to finish
instructions = open("2016/12.txt").read().strip().splitlines()
while pc < len(instructions):
    match instructions[pc].split():
        case "cpy", x, y:
            reg[y] = value(x)
        case "inc", x:
            reg[x] += 1
        case "dec", x:
            reg[x] -= 1
        case "jnz", x, y:
            if value(x) != 0:
                pc = pc + int(y) - 1
    pc += 1
print(f'a: {reg["a"]}')  # 318003

a: 318003


In [57]:
# Day 13: A Maze of Twisty Little Cubicles
def is_wall(x: int, y: int, fav: int) -> bool:
    num = x * x + 3 * x + 2 * x * y + y + y * y + fav
    return bin(num).count("1") % 2


def in_bounds(x: int, y: int) -> bool:
    return 0 <= x and 0 <= y


def valid_paths(x: int, y: int, fav: int):
    candidates = [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]
    return [
        c for c in candidates if in_bounds(c[0], c[1]) and not is_wall(c[0], c[1], fav)
    ]


def steps_to(goal_x: int, goal_y: int, fav: int) -> int:
    frontier = []
    heappush(frontier, (0, (1, 1)))  # We start at 1,1
    cost_so_far = defaultdict(lambda: inf)
    cost_so_far[(1, 1)] = 0

    while frontier:
        x, y = heappop(frontier)[1]

        if (x, y) == (goal_x, goal_y):
            break

        for (next_x, next_y) in valid_paths(x, y, fav):
            new_cost = cost_so_far[(x, y)] + 1
            if new_cost < cost_so_far[(next_x, next_y)]:
                cost_so_far[(next_x, next_y)] = new_cost
                heappush(frontier, (new_cost, (next_x, next_y)))

    return cost_so_far


locations = steps_to(31, 39, 1362)

print(f"Part 1: {locations[(31,39)]}")  # 82
print(f"Part 2: {sum(1 for dist in locations.values() if dist <= 50)}")

Part 1: 82
Part 2: 138


In [24]:
# Day 14: One-Time Pad
def triplet(text: str) -> str:
    for a, b, c in zip(text, text[1:], text[2:]):
        if a == b == c:
            return a
    return False


def contains_five(char: str, text: str) -> str:
    return char * 5 in text


@lru_cache(maxsize=1001)
def hashval(salt: str, index: int, stretch=0) -> str:
    hash = salt + str(index)
    hash = hashlib.md5(hash.encode("utf-8")).hexdigest()
    for _ in range(stretch):
        hash = hashlib.md5(hash.encode("utf-8")).hexdigest()
    return hash


def calculate_index(salt: str, stretch=0):
    index = 0
    matches = 0
    while matches < 64:
        if char := triplet(hashval(salt, index, stretch)):
            for offset in range(1, 1001):
                if contains_five(char, hashval(salt, index + offset, stretch)):
                    matches += 1
                    break
        index += 1
    return index - 1


print(f'Part 1: {calculate_index("cuanljph")}')  # 23769
print(f'Part 2: {calculate_index("cuanljph", stretch=2016)}')  # 20606

Part 1: 23769
CPU times: user 972 ms, sys: 4.56 ms, total: 977 ms
Wall time: 980 ms


In [17]:
# Day 15: Timing is Everything
match = "Disc #{:d} has {:d} positions; at time={:d}, it is at position {:d}."
lines = open("2016/15.txt").read().strip().splitlines()
discs = [parse(match, line).fixed for line in lines]

time = 0
while sum((cur + offset + time) % positions for (offset, positions, _, cur) in discs):
    time += 1
print(f"Part 1: {time}")

discs.append((7, 11, 0, 0))
time = 0
while sum((cur + offset + time) % positions for (offset, positions, _, cur) in discs):
    time += 1
print(f"Part 2: {time}")

Part 1: 376777
Part 2: 3903937


In [54]:
# Day 16: Dragon Checksum
def fold(a: str) -> str:
    """
    Call the data you have at this point "a".
    Make a copy of "a"; call this copy "b".
    Reverse the order of the characters in "b".
    In "b", replace all instances of 0 with 1 and all 1s with 0.
    The resulting data is "a", then a single 0, then "b".
    """

    def flip(text: str) -> str:
        return text.translate(str.maketrans("01", "10"))

    b = flip(a[::-1])
    return a + "0" + b


assert fold("1") == "100"
assert fold("111100001010") == "1111000010100101011110000"


def checksum(data: str) -> str:
    """
    The checksum for some given data is created by considering each non-overlapping pair
    of characters in the input data. If the two characters match (00 or 11), the next
    checksum character is a 1. If the characters do not match (01 or 10), the next
    checksum character is a 0. This should produce a new string which is exactly half as
    long as the original. If the length of the checksum is even, repeat the process until
    you end up with a checksum with an odd length.
    """
    while len(data) % 2 == 0:
        data = "".join(
            ("1" if data[i] == data[i + 1] else "0") for i in range(0, len(data), 2)
        )
    return data


assert checksum("110010110100") == "100"


def fill_disk(disk: int, seed: str) -> str:
    while len(seed) < disk:
        seed = fold(seed)
    return seed[:disk]


disk = 272
initial = "10111100110001111"
print(f"Part 1: {checksum(fill_disk(disk, initial))}")

disk = 35651584
print(f"Part 2: {checksum(fill_disk(disk, initial))}")

Part 1: 11100110111101110
Part 2: 10001101010000101


In [35]:
# Day 17: Two Steps Forward
def in_bounds(point: Tuple[int, int]) -> bool:
    x, y = point
    return 0 <= x <= 3 and 0 <= y <= 3


def open_doors(keys: str) -> str:
    """Only the first four characters of the hash are used; they represent, respectively,
    the doors up, down, left, and right from your current position. Any b, c, d, e, or f
    means that the corresponding door is open; any other character (any number or a) means
    that the corresponding door is closed and locked."""
    valid_keys = "bcdef"
    doors = ""
    if keys[0] in valid_keys:
        doors += "U"
    if keys[1] in valid_keys:
        doors += "D"
    if keys[2] in valid_keys:
        doors += "L"
    if keys[3] in valid_keys:
        doors += "R"
    return doors


def generate_keys(text: str) -> str:
    "Four first characters in MD5 hash of text"
    return hashlib.md5(text.encode("utf-8")).hexdigest()[:4]


assert open_doors("ced9") == "UDL"
assert generate_keys("hijkl") == "ced9"


def neighbors(position: Tuple[str, int, int], seed: str) -> List[Tuple[str, int, int]]:
    "Return possible moves from point and where you end up if you move in that direction."
    path, x, y = position
    directions = open_doors(generate_keys(seed + path))
    rooms = []
    for dir in directions:
        match dir:
            case "U":
                rooms.append((path + dir, x - 1, y))
            case "D":
                rooms.append((path + dir, x + 1, y))
            case "L":
                rooms.append((path + dir, x, y - 1))
            case "R":
                rooms.append((path + dir, x, y + 1))
    return [room for room in rooms if in_bounds((room[1], room[2]))]


def shortest_path(seed: str):
    frontier = []
    start = ("", 0, 0)
    heappush(frontier, (0, start))

    while frontier:
        current = heappop(frontier)[1]
        path, x, y = current

        if (x, y) == (3, 3):
            break

        for next in neighbors(current, seed):
            priority = len(path) + 1  # Cost to move to next node
            heappush(frontier, (priority, next))

    return path


def longest_path(position: Tuple[str, int, int], seed: str) -> int:
    "DP solution"
    path, x, y = position
    if (x, y) == (3, 3):
        return len(path)

    worst = 0
    for move in neighbors(position, seed):
        new_cost = longest_path(move, seed)
        if new_cost > worst:
            worst = new_cost

    return worst


def bfs(start: Tuple[str, int, int], seed: str) -> List[str]:
    "Returns all paths, alternative to above"
    queue = [start]
    while queue:
        current = queue.pop()
        for move in neighbors(current, seed):
            path, x, y = move
            if (x, y) == (3, 3):
                yield path
            else:
                queue.append((move))


def dfs(start, seed) -> int:
    "Find the longest path to goal by depth-first search."
    longest = 0
    frontier = [start]
    while frontier:
        state = (path, x, y) = frontier.pop()
        if (x, y) == (3, 3):
            longest = max(longest, len(path))
        else:
            frontier.extend(neighbors(state, seed))
    return longest


print(f'Part 1: {shortest_path("pxxbnzuo")}')  # RDULRDDRRD
print(f'Part 2: {longest_path(("", 0, 0), "pxxbnzuo")}')  # 752
# paths = list(bfs(("", 0, 0), "pxxbnzuo"))
# print("Part 1:", paths[0])
# print("Part 2:", max(len(path) for path in paths))
# dfs(("", 0, 0), "pxxbnzuo")

Part 1: RDULRDDRRD
Part 2: 752


In [94]:
# Day 18: Like a Rogue
safe, trap = ".", "^"


def new_row(row: str) -> str:
    prev = safe + row + safe
    new = ""
    for left, right in zip(prev, prev[2:]):
        new += trap if left != right else safe
    return new


def safe_tiles(row: str, iterations: int) -> int:
    rows = []
    for _ in range(iterations):
        rows.append(row)
        row = new_row(row)
    return "".join(rows).count(safe)


row = open("2016/18.txt").read().strip()
print("Part 1:", safe_tiles(row, 40))  # 1961
print("Part 2:", safe_tiles(row, 400000))

Part 1: 1961
Part 2: 20000795


In [106]:
# Day 19: An Elephant Named Joseph
N = 3014603
elves = deque(i for i in range(1, N + 1))
while len(elves) > 1:
    elves.rotate(-1)
    elves.popleft()
print("Part 1:", elves[0])  # 1834903, ~1 second

# Part 2:
# This one takes 90 minutes (!!) to complete, but gives the right answer 1420280
# N = 3014603
# N = 5
# elfs = deque(i for i in range(1, N + 1))
# while len(elfs) > 1:
#     print(elfs)
#     del elfs[len(elfs) // 2]
#     elfs.rotate(-1)
#     if len(elfs) % 100_000 == 0:
#         print(len(elfs))
# print(elfs[0])  # 1420280

# Same principle, but use two deques for fast removal at end (which is middle of data
# structure). Runs in 2 seconds
left = deque(range(1, N // 2 + 1))
right = deque(range(N // 2 + 1, N + 1))

while left and right:
    # print(left, right)
    if len(left) <= len(right):
        right.popleft()
    else:
        left.pop()

    # Rotate
    try:
        left.append(right.popleft())
        right.append(left.popleft())
    except IndexError:
        # Last removal will pop from an empty deque
        pass
print("Part 2:", left[0])

Part 1: 1834903
Part 2: 1420280


In [28]:
# Day 20: Firewall rules
def lowest_ip(ips):
    for (prev_low, prev_high), (cur_low, cur_high) in zip(ips, ips[1:]):
        if cur_low > prev_high + 1:
            return prev_high + 1


def replace_overlaps(ips):
    result = []
    for a, b in zip(ips, ips[1:]):
        low_a, high_a = a
        low_b, high_b = b
        if low_b <= high_a + 1:
            # a and b overlaps, replace with a larger range
            result.append((min(low_a, low_b), max(high_a, high_b)))
        else:
            result.append(a)
    result.append(b)
    return result


def remove_enclosed(ips):
    result = ips.copy()
    for a, b in zip(ips, ips[1:]):
        low_a, high_a = a
        low_b, high_b = b
        if low_a <= low_b and high_a >= high_b:
            # print(f'{a} encloses {b}')
            result.remove(b)
    return result


def reduce_ip_ranges(ips):
    "Reduce to non overlapping sequences"
    items = len(ips)
    while True:
        ips = replace_overlaps(ips)
        ips = remove_enclosed(ips)
        if items == len(ips):
            break
        else:
            items = len(ips)
    return ips


def allowed_ips(ips, N):
    ips = reduce_ip_ranges(ips)
    blocked = 0
    for low, high in ips:
        blocked += high - low + 1
    return N - blocked + 1


lines = open("2016/20.txt").read().strip().splitlines()
ips = [(int(low), int(high)) for low, high in (line.split("-") for line in lines)]
ips.sort()
N = 4294967295

print("Part 1:", lowest_ip(ips))  # 17348574
print("Part 2:", allowed_ips(ips, N))  # 104

Part 1: 17348574
Part 2: 104


In [32]:
# Peter Norvig's solution for day 20. Elegant.
def first(iterable):
    return next(iter(iterable))


def unblocked(pairs):
    "Find the lowest unblocked integer, given the sorted pairs of blocked numbers."
    i = 0
    for (low, high) in pairs:
        yield from range(i, low)
        i = max(i, high + 1)


ips = [(int(low), int(high)) for low, high in (line.split("-") for line in lines)]
ips.sort()

print("Part 1:", first(unblocked(ips)))
print("Part 2:", len(list(unblocked(ips))))

Part 1: 17348574
Part 2: 104


In [194]:
# Day 21: Scrambled Letter and Hash
def scramble(pw, lines):
    def rotate(text, steps):
        "Positive steps rotates right, negative rotates left"
        text = deque(text)
        text.rotate(steps)
        return list(text)

    def rotate_position(text, letter):
        """
        Rotate based on index of letter.

        Once the index is determined, rotate the string to the right one time, plus a
        number of times equal to that index, plus one additional time if the index was
        at least 4.
        """
        steps = text.index(letter)
        if steps >= 4:
            steps += 1
        steps += 1
        return rotate(text, steps)

    pw = list(pw)

    for line in lines:
        if p := parse("swap position {:d} with position {:d}", line):
            a, b = p
            pw[a], pw[b] = pw[b], pw[a]
        elif p := parse("swap letter {} with letter {}", line):
            a, b = p
            a, b = pw.index(a), pw.index(b)
            pw[a], pw[b] = pw[b], pw[a]
        elif p := parse("reverse positions {:d} through {:d}", line):
            a, b = p
            pw[a : b + 1] = pw[a : b + 1][::-1]
        elif p := parse("rotate left 1 step", line):
            pw = rotate(pw, -1)
        elif p := parse("rotate left {:d} steps", line):
            a = p[0]
            pw = rotate(pw, -a)
        elif p := parse("rotate right 1 step", line):
            pw = rotate(pw, 1)
        elif p := parse("rotate right {:d} steps", line):
            a = p[0]
            pw = rotate(pw, a)
        elif p := parse("move position {:d} to position {:d}", line):
            a, b = p
            pw.insert(b, pw.pop(a))
        elif p := parse("rotate based on position of letter {}", line):
            a = p[0]
            pw = rotate_position(pw, a)
        else:
            raise ValueError("Can't parse", line)
    return pw


def descramble(target, lines):
    target = list(target)
    for combo in permutations(target, len(target)):
        if scramble(combo, lines) == target:
            return combo


# Runs in 1m 10s
lines = open("2016/21.txt").read().strip().splitlines()
print("Part 1:", "".join(scramble("abcdefgh", lines)))  # gbhafcde
pw2 = descramble("fbgdceah", lines)
print("Part 2:", "".join(pw2))  # bcfaegdh

Part 1: gbhafcde
Part 2: bcfaegdh


In [3]:
# Day 22: Grid Computing
def parse_ints(line):
    return [int(x) for x in re.findall("\d+", line)]


class Node(NamedTuple):
    size: int
    used: int
    avail: int
    use: int


def viable_pair(a, b):
    return all((a.used > 0, a is not b, a.used <= b.avail))


lines = open("2016/22.txt").read().strip().splitlines()
nodes = {}

for line in lines[2:]:
    x, y, size, used, avail, use = parse_ints(line)
    nodes[(x, y)] = Node(size, used, avail, use)

part1 = sum(viable_pair(nodes[a], nodes[b]) for (a, b) in permutations(nodes, 2))
print(f"Part 1: {part1}")  # 967

# Part 2 - manual solve
# for point, node in nodes.items():
#     if point[1] == 0:
#         print(f"\n{point[0]:2d}", end=" ")
#     # nodes between 0 and 100 are possible to shift with empty slot.
#     char = "." if 0 < node.size < 100 else "#"
#     if node.used == 0:
#         char = "X"
#     if point == (0, 0):
#         char = "D"
#     if point == (32, 0):
#         char = "G"
#     print(char, end="")

# # Move X to spot above G, then 5 moves for each step G and X
# # moves up together and a final step to get G in place.
# # See 2016-day22.txt for details.
# print("\nPart 2", 16 + 12 + 21 + 31 * 5 + 1)  # 205

# Part 2 with A*
def a_star(start):
    frontier = []
    heappush(frontier, (0, start))
    came_from = {}
    cost_so_far = defaultdict(lambda: inf)
    came_from[start] = None
    cost_so_far[start] = 0

    while frontier:
        current = heappop(frontier)[1]

        if heuristic(current) == 0:
            return came_from, cost_so_far, current

        for next in moves(current):
            new_cost = cost_so_far[current] + cost(current, next)
            if new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost + heuristic(next)
                heappush(frontier, (priority, next))
                came_from[next] = current


def heuristic(current) -> float:
    "Distance between data and goal"
    (x1, y1) = current.data
    (x2, y2) = (0, 0)
    return abs(x1 - x2) + abs(y1 - y2)


def cost(current, next):
    return 1


def moves(current):
    "Possible coordinates that empty can move to."
    x, y = current.empty
    candidates = [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]
    valid = [
        point for point in candidates if point in nodes and nodes[point].used < 100
    ]
    for v in valid:
        if v == current.data:
            yield State(empty=v, data=current.empty)
        else:
            yield State(empty=v, data=current.data)


class State(NamedTuple):
    empty: Tuple[int, int]
    data: Tuple[int, int]


def X(point):
    return point[0]


empty = next(iter(point for (point, node) in nodes.items() if node.used == 0))
data = (max(X(point) for point in nodes), 0)
start = State(empty, data)
came_from, cost_so_far, current = a_star(start)
print("Part 2:", cost_so_far[current])

Part 1: 967
Part 2: 205


In [45]:
# Day 23: Safe Cracking
def value(x):
    try:
        return int(x)
    except ValueError:
        return reg[x]


def toggle(instruction: str) -> str:
    match instruction.split():
        case "inc", x:
            return "dec " + x
        case "dec", x:
            return "inc " + x
        case "tgl", x:
            return "inc " + x
        case "cpy", x, y:
            return "jnz " + x + " " + y
        case "jnz", x, y:
            return "cpy " + x + " " + y
        case _:
            raise ValueError("Cant parse", instruction)


pc = 0
reg = {"a": 7, "b": 0, "c": 0, "d": 0}
instructions = open("2016/23.txt").read().strip().splitlines()
while pc < len(instructions):
    match instructions[pc].split():
        case "cpy", x, y:
            reg[y] = value(x)
        case "inc", x:
            reg[x] += 1
        case "dec", x:
            reg[x] -= 1
        case "jnz", x, y:
            if value(x) != 0:
                pc = pc + value(y) - 1
        case "tgl", x:
            try:
                instructions[pc + value(x)] = toggle(instructions[pc + value(x)])
                print(
                    f"Successful toggle {instructions[pc + value(x)]} to {toggle(instructions[pc + value(x)])}"
                )
                print(f"Registers: {reg}")
            except IndexError:
                # Attempt to toggle instruction outside the program. Nothing happens
                print(f"Failed toggle at {pc + value(x)}")
                print(f"Registers: {reg}")
                pass

    pc += 1
print(f'Part 1: {reg["a"]}')  # 10152

print(
    """
Part 2: what should the value be for a=12. Looking at the registers during execution gives
a factorial pattern of a. The final result gives an offset to the factorial

a: 5 does not finish 'soon'
a: 6 -> 5832   6! = 720
a: 7 -> 10152  7! = 5040
a: 8 -> 45432  8! = 40320

Result for a=12 should be 12! + 5112 = 479006712 - and that is correct.
"""
)

Failed toggle at 26
Registers: {'a': 42, 'b': 5, 'c': 10, 'd': 0}
Successful toggle dec c to inc c
Registers: {'a': 210, 'b': 4, 'c': 8, 'd': 0}
Successful toggle dec d to inc d
Registers: {'a': 840, 'b': 3, 'c': 6, 'd': 0}
Successful toggle cpy 72 d to jnz 72 d
Registers: {'a': 2520, 'b': 2, 'c': 4, 'd': 0}
Successful toggle cpy 1 c to jnz 1 c
Registers: {'a': 5040, 'b': 1, 'c': 2, 'd': 0}
Part 1: 10152

Part 2: what should the value be for a=12. Looking at the registers during execution gives
a factorial pattern of a. The final result gives an offset to the factorial

a: 5 does not finish 'soon'
a: 6 -> 5832   6! = 720
a: 7 -> 10152  7! = 5040
a: 8 -> 45432  8! = 40320

Result for a=12 should be 12! + 5112 = 479006712 - and that is correct.



In [41]:
# Day 24: Air Duct Spelunking
def a_star(start):
    """
    A* search from state `start`.
    Need to provide functions: `heuristic`, `cost`, and `moves`
    """
    frontier = []
    heappush(frontier, (0, start))
    cost_so_far = defaultdict(lambda: inf)
    cost_so_far[start] = 0

    while frontier:
        current = heappop(frontier)[1]
        if heuristic(current) == 0:
            break
        for next in moves(current):
            new_cost = cost_so_far[current] + cost(current, next)
            if new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost + heuristic(next)
                heappush(frontier, (priority, next))
    return cost_so_far, current


def cost(current, next):
    return 1


def moves(current):
    row, column = current.row, current.column
    candidates = [
        (row - 1, column),
        (row + 1, column),
        (row, column - 1),
        (row, column + 1),
    ]
    for (r, c) in candidates:
        goals_visited = current.goals_visited
        if grid[r][c] in goals:
            goals_visited = goals_visited | frozenset(grid[r][c])
        if grid[r][c] != "#":
            yield GridState(r, c, goals_visited)


def heuristic(state):
    "Number of goals left to visit."
    # Consider adding distance to closest goal, might improve performance if needed
    return len(goals) - len(state.goals_visited)


class GridState(NamedTuple):
    "Position in grid and achieved goals"
    row: int
    column: int
    goals_visited: FrozenSet[str]


@cache
def start():
    "Return GridState for starting point"
    row, column = next(
        iter(
            (r, c)
            for r in range(len(grid))
            for c in range(len(grid[r]))
            if grid[r][c] == "0"
        )
    )
    return GridState(row, column, frozenset())


grid = open("2016/24.txt").read().strip().splitlines()
goals = frozenset(x for line in grid for x in re.findall(r"[1-9]", line))

cost_so_far, current = a_star(start())
collect_all = cost_so_far[current]
print("Part 1:", collect_all)  # 470

# For Part 2 we need to get back to start from current position. Redefine heuristics
def heuristic(state):
    "Distance to start"
    return abs(state.row - start().row) + abs(state.column - start().column)


cost_so_far, current = a_star(current)
get_back = cost_so_far[current]
print("Part 2:", collect_all + get_back)

Part 1: 470
Part 2: 720


In [30]:
# Day 25 - Clock Signal
def clock_signals(a=0, steps=10**6):
    def value(x):
        return reg[x] if x in reg else int(x)

    pc = 0
    reg = {"a": a, "b": 0, "c": 0, "d": 0}
    instructions = open("2016/25.txt").read().strip().splitlines()
    for _ in range(steps):
        if not (0 <= pc < len(instructions)):
            return
        match instructions[pc].split():
            case "cpy", x, y:
                reg[y] = value(x)
            case "inc", x:
                reg[x] += 1
            case "dec", x:
                reg[x] -= 1
            case "jnz", x, y:
                if value(x) != 0:
                    pc = pc + value(y) - 1
            case "out", x:
                yield value(x)
        pc += 1


def repeats(a, repeats=100):
    targets = cycle((0, 1))
    for (i, (target, signal)) in enumerate(zip(targets, clock_signals(a))):
        if signal != target:
            return False
        if i >= repeats:
            return True


print("Part 1:", next(iter(a for a in range(10**100) if repeats(a))))

Part 1: 198
